# Part 1

In [1]:
class Point:
    def __init__(self, x, y, dx, dy):
        self.x = x
        self.y = y
        self.dx = dx
        self.dy = dy
    
    def update(self):
        self.x += self.dx
        self.y += self.dy
    
    def __repr__(self):
        return 'Point<x={},y={},dx={},dy={}>'.format(self.x, self.y, self.dx, self.dy)

In [2]:
import re

point_re = re.compile(r'^position=<\s*(-?\d+),\s*(-?\d+)> velocity=<\s*(-?\d+),\s*(-?\d+)>$')

def parse_points(lines):
    ''' Parse input lines into a list of points. '''
    points = list()
    for line in lines:
        match = point_re.match(line)
        if not match:
            raise Exception('Could not parse: {}'.format(line))
        x,y = int(match.group(1)), int(match.group(2))
        dx,dy = int(match.group(3)), int(match.group(4))
        points.append(Point(x, y, dx, dy))
    return points

In [3]:
test_text = '''position=< 9,  1> velocity=< 0,  2>
position=< 7,  0> velocity=<-1,  0>
position=< 3, -2> velocity=<-1,  1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2,  2>
position=<-6, 10> velocity=< 2, -2>
position=< 1,  8> velocity=< 1, -1>
position=< 1,  7> velocity=< 1,  0>
position=<-3, 11> velocity=< 1, -2>
position=< 7,  6> velocity=<-1, -1>
position=<-2,  3> velocity=< 1,  0>
position=<-4,  3> velocity=< 2,  0>
position=<10, -3> velocity=<-1,  1>
position=< 5, 11> velocity=< 1, -2>
position=< 4,  7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0,  1>
position=<15,  0> velocity=<-2,  0>
position=< 1,  6> velocity=< 1,  0>
position=< 8,  9> velocity=< 0, -1>
position=< 3,  3> velocity=<-1,  1>
position=< 0,  5> velocity=< 0, -1>
position=<-2,  2> velocity=< 2,  0>
position=< 5, -2> velocity=< 1,  2>
position=< 1,  4> velocity=< 2,  1>
position=<-2,  7> velocity=< 2, -2>
position=< 3,  6> velocity=<-1, -1>
position=< 5,  0> velocity=< 1,  0>
position=<-6,  0> velocity=< 2,  0>
position=< 5,  9> velocity=< 1, -2>
position=<14,  7> velocity=<-2,  0>
position=<-3,  6> velocity=< 2, -1>'''

In [4]:
test_points = parse_points(test_text.split('\n'))
print(len(test_points))
print(test_points[:5])

31
[Point<x=9,y=1,dx=0,dy=2>, Point<x=7,y=0,dx=-1,dy=0>, Point<x=3,y=-2,dx=-1,dy=1>, Point<x=6,y=10,dx=-2,dy=-1>, Point<x=2,y=-4,dx=2,dy=2>]


In [5]:
def create_matrix(points):
    ''' Return a sparse matrix as a set, where each element (x,y) represents a
    point. Also returns min and max x and y values. '''
    matrix = set()
    for point in points:
        matrix.add((point.x, point.y))
    return matrix

In [6]:
test_mat = create_matrix(test_points)
print(len(test_mat))

31


In [7]:
def print_matrix(matrix):
    ''' Render a matrix. '''
    min_x, min_y =  99999999,  99999999
    max_x, max_y = -99999999, -99999999
    for (x, y) in matrix:
        if x < min_x:
            min_x = x
        elif x > max_x:
            max_x = x
        if y < min_y:
            min_y = y
        elif y > max_y:
            max_y = y
    for y in range(min_y, max_y + 1):
        print(''.join('#' if (x,y) in matrix else '.' for x in range(min_x, max_x + 1)))

In [8]:
print_matrix(test_mat)

........#.............
................#.....
.........#.#..#.......
......................
#..........#.#.......#
...............#......
....#.................
..#.#....#............
.......#..............
......#...............
...#...#.#...#........
....#..#..#.........#.
.......#..............
...........#..#.......
#...........#.........
...#.......#..........


In [9]:
def play_round(points):
    ''' Update all points and print result. Note: this modifies the points passed in! '''
    for point in points:
        point.update()

In [10]:
# The next cell modifies test_points, so we have a cell here to recreate it.
test_points = parse_points(test_text.split('\n'))
print(len(test_points))

31


In [13]:
# This cell can be run multiple times to animate the map.
# I ran it just enough times to produce the message.
play_round(test_points)
print_matrix(create_matrix(test_points))

#...#..###
#...#...#.
#...#...#.
#####...#.
#...#...#.
#...#...#.
#...#...#.
#...#..###


In [14]:
def detect_vertical(matrix):
    ''' I think the best way to recognize end state is to look for 8 # in a column,
    since probably at least one letter will have this feature. The sample message HI
    has three of them! '''
    matrix = set(matrix)
    while matrix:
        x, y = matrix.pop()
        count = 1
        # Go up
        while (x, y-1) in matrix:
            y -= 1
            matrix.remove((x,y))
            count += 1
        # Go down
        while (x, y+1) in matrix:
            y += 1
            matrix.remove((x,y))
            count += 1
        if count == 8:
            return True
    return False

In [15]:
test_mat = create_matrix(test_points)
detect_vertical(test_mat)

True

In [16]:
play_round(test_points)
test_mat = create_matrix(test_points)
detect_vertical(test_mat)

False

In [17]:
with open('input.txt') as input_:
    points = parse_points([line for line in input_])
print(len(points))

369


In [41]:
# The original matrix is too big to display. Hopefully when the message appears
# it is small enough to display. Run this cell a few times until it finds a mesage.
for _ in range(5000):
    play_round(points)
    mat = create_matrix(points)
    if detect_vertical(mat):
        print('Found it?')
        break
else:
    print("Didn't find it")

Found it?


In [42]:
# Probably took 25k iterations of more of the above. Did it work?
print_matrix(mat)

.####...######....##....#....#..######..#....#..#....#.....###
#....#..#........#..#...##...#..#.......#....#..#...#.......#.
#.......#.......#....#..##...#..#.......#....#..#..#........#.
#.......#.......#....#..#.#..#..#.......#....#..#.#.........#.
#.......#####...#....#..#.#..#..#####...######..##..........#.
#..###..#.......######..#..#.#..#.......#....#..##..........#.
#....#..#.......#....#..#..#.#..#.......#....#..#.#.........#.
#....#..#.......#....#..#...##..#.......#....#..#..#....#...#.
#...##..#.......#....#..#...##..#.......#....#..#...#...#...#.
.###.#..#.......#....#..#....#..######..#....#..#....#...###..


# Part 2

In [47]:
# Can reuse code from above, just need to keep track of how many iterations
# are used.
test_points = parse_points(test_text.split('\n'))
print(len(test_points))

31


In [48]:
import itertools

for n in itertools.count():
    play_round(test_points)
    test_mat = create_matrix(test_points)
    if detect_vertical(test_mat):
        print('Found it?')
        break

Found it?


In [49]:
print_matrix(test_mat)

#...#..###
#...#...#.
#...#...#.
#####...#.
#...#...#.
#...#...#.
#...#...#.
#...#..###


In [50]:
# Note this is one less than the total number, because itertools.count() started
# at zero.
n

2

In [53]:
with open('input.txt') as input_:
    points = parse_points([line for line in input_])
print(len(points))

369


In [54]:
for n in itertools.count():
    play_round(points)
    mat = create_matrix(points)
    if detect_vertical(mat):
        print('Found it?')
        break

Found it?


In [55]:
print_matrix(mat)

.####...######....##....#....#..######..#....#..#....#.....###
#....#..#........#..#...##...#..#.......#....#..#...#.......#.
#.......#.......#....#..##...#..#.......#....#..#..#........#.
#.......#.......#....#..#.#..#..#.......#....#..#.#.........#.
#.......#####...#....#..#.#..#..#####...######..##..........#.
#..###..#.......######..#..#.#..#.......#....#..##..........#.
#....#..#.......#....#..#..#.#..#.......#....#..#.#.........#.
#....#..#.......#....#..#...##..#.......#....#..#..#....#...#.
#...##..#.......#....#..#...##..#.......#....#..#...#...#...#.
.###.#..#.......#....#..#....#..######..#....#..#....#...###..


In [56]:
n + 1

10086